In [25]:
from code.neuron import *
from code.encodingsource import *
from code.hsgs import *
from code.classical_neuron import *
from code.classical_pso import *
from code.sf import *
simulator = Aer.get_backend('qasm_simulator')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import time

from code.experiment_functions import *

In [2]:
X_train = [[-1, -1],  [-1,1], [1,1], [1,-1]]
y_train = [1, 0, 1, 0]

X_test =  [[-1, -1],  [-1,1], [1,1], [1,-1]]
y_test =  [1, 0, 1, 0]

## Test

In [3]:
w = list(np.random.uniform(low=0.0, high=1.0, size=(len(X_test[0]),)))

In [4]:
weights = quantumNeuronFIT(X_train, 
                           y_train,
                           init_weight = w,
                           threshold=0.7,
                           lrParameter=0.1, 
                           n_epochs=1, 
                           trainingBias=False,
                           epoch_results=False,
                           phaseEstrategyOperator='original',
                           trainingApproaches={'hsgs'})

lrParameter:  0.1
threshold:  0.7
trainingBias:  False
phaseEstrategyOperator:  original
best error HSGS training:  0


In [5]:
results = quantumNeuronPREDICT(X_test, 
                                   y_test,
                                   threshold=0.7,
                                   weightVectorsPhaseEncoding = weights[0], 
                                   weightVectorsHSGS = weights[1], 
                                   repeat=1,
                                   bias=False, 
                                   phaseEstrategyOperator='original',
                                   testingApproaches={'hsgs'})

## Binary Quantum Neuron Experiments 

In [11]:
output = {'model':[],
          'phase_strategy':[],
          'threshold':[],
            'bias':[],
            'lr':[],
            'initial_weights':[],
            'neuron_outputs':[],
            'circuit_size':[]
         }

In [12]:
# search space
initial_weights =  list(np.random.uniform(low=0.0, high=1.0, size=(len(X_train[0]),)))
threshold_space = [0.3, 0.5, 0.7, 0.8, 0.9]
lr_space = [0.02, 0.1]
bias_space = [False, True]
strategy_space = ['original']

In [13]:
%%time
for w in initial_weights:
    for strategy in strategy_space:
        for threshold in threshold_space:
            for lr in lr_space:
                for bias in bias_space:
                    # execute experiment
                    weights = quantumNeuronFIT(X_train, 
                                               y_train,
                                               init_weight=initial_weights,
                                               threshold=threshold,
                                               lrParameter=lr, 
                                               n_epochs=20, 
                                               trainingBias=bias,
                                               phaseEstrategyOperator=strategy,
                                               trainingApproaches={'hsgs'})

                    results = quantumNeuronPREDICT(X_test, 
                                                   y_test,
                                                   threshold=threshold,
                                                   weightVectorsPhaseEncoding = weights[0], 
                                                   weightVectorsHSGS = weights[1], 
                                                   repeat=5,
                                                   bias=bias, 
                                                   phaseEstrategyOperator=strategy,
                                                   testingApproaches={'hsgs'})


                    output['model'].append('HSGS')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['neuron_outputs'].append(results[1])
                    output['circuit_size'].append(results[3])
                    output['initial_weights'].append(w)
                    print('\n')

lrParameter:  0.02
threshold:  0.3
trainingBias:  False
phaseEstrategyOperator:  original
best error HSGS training:  0


lrParameter:  0.02
threshold:  0.3
trainingBias:  True
phaseEstrategyOperator:  original
best error HSGS training:  1


lrParameter:  0.1
threshold:  0.3
trainingBias:  False
phaseEstrategyOperator:  original
best error HSGS training:  0


lrParameter:  0.1
threshold:  0.3
trainingBias:  True
phaseEstrategyOperator:  original
best error HSGS training:  1


lrParameter:  0.02
threshold:  0.5
trainingBias:  False
phaseEstrategyOperator:  original
best error HSGS training:  0


lrParameter:  0.02
threshold:  0.5
trainingBias:  True
phaseEstrategyOperator:  original
best error HSGS training:  1


lrParameter:  0.1
threshold:  0.5
trainingBias:  False
phaseEstrategyOperator:  original
best error HSGS training:  0


lrParameter:  0.1
threshold:  0.5
trainingBias:  True
phaseEstrategyOperator:  original
best error HSGS training:  1


lrParameter:  0.02
threshold:  0.7
train

In [14]:
experiment_hsgs = pd.DataFrame(output)

In [16]:
experiment_hsgs.to_csv('data_and_results/XOR/XOR_probs_hsgs_size.csv', index=False)

## Continuously Valued Quantum Neuron Experiments

In [43]:
output = {'model':[],
          'phase_strategy':[],
            'bias':[],
            'threshold':[],
            'lr':[],
            'initial_weights':[],
            'neuron_outputs':[],
            'circuit_size':[]
         }

In [44]:
# search space
initial_weights =  list(np.random.uniform(low=0.0, high=1.0, size=(len(X_test[0]),)))
threshold_space = [0.3, 0.5, 0.7, 0.8, 0.9]
lr_space = [0.02, 0.1]
bias_space = [False, True]
strategy_space = ['original', 'angle', 'radius', 'angleradius']
# get time of execution
time_exec = []

In [45]:
%%time
for strategy in strategy_space:
    start_time = time.time()
    for threshold in threshold_space:
        for lr in lr_space:
            for bias in bias_space:
                # execute experiment
                weights = quantumNeuronFIT(X_train, 
                                           y_train,
                                           init_weight=initial_weights,
                                           threshold=threshold,
                                           lrParameter=lr, 
                                           n_epochs=20, 
                                           trainingBias=bias,
                                           phaseEstrategyOperator=strategy,
                                           trainingApproaches={'phase-encoding'})

                results = quantumNeuronPREDICT(X_test, 
                                                   y_test,
                                                   threshold=threshold,
                                                   weightVectorsPhaseEncoding = weights[0], 
                                                   weightVectorsHSGS = weights[1], 
                                                   repeat=5,
                                                   bias=bias, 
                                                   phaseEstrategyOperator=strategy,
                                                   testingApproaches={'phase-encoding'})

                # get experiment results
                output['model'].append('phase-encoding')
                output['lr'].append(lr)
                output['bias'].append(bias)
                output['threshold'].append(threshold)
                output['phase_strategy'].append(strategy)
                output['neuron_outputs'].append(results[0])
                output['circuit_size'].append(results[2])
                output['initial_weights'].append(w)
                print('\n')
    print('\nTIME OF EXECUTION ', strategy, ': ', time.time() - start_time,' seconds\n')
    time_exec.append(time.time() - start_time)

lrParameter:  0.02
threshold:  0.3
trainingBias:  False
phaseEstrategyOperator:  original

best error phase-encoding training:  2


lrParameter:  0.02
threshold:  0.3
trainingBias:  True
phaseEstrategyOperator:  original

best error phase-encoding training:  2


lrParameter:  0.1
threshold:  0.3
trainingBias:  False
phaseEstrategyOperator:  original

best error phase-encoding training:  2


lrParameter:  0.1
threshold:  0.3
trainingBias:  True
phaseEstrategyOperator:  original

best error phase-encoding training:  2


lrParameter:  0.02
threshold:  0.5
trainingBias:  False
phaseEstrategyOperator:  original

best error phase-encoding training:  2


lrParameter:  0.02
threshold:  0.5
trainingBias:  True
phaseEstrategyOperator:  original

best error phase-encoding training:  2


lrParameter:  0.1
threshold:  0.5
trainingBias:  False
phaseEstrategyOperator:  original

best error phase-encoding training:  2


lrParameter:  0.1
threshold:  0.5
trainingBias:  True
phaseEstrategyOperator:  ori

In [46]:
experiment_phase = pd.DataFrame(output)

In [47]:
experiment_phase

model phase_strategy   bias  threshold    lr  initial_weights  \
0   phase-encoding       original  False        0.3  0.02         0.538496   
1   phase-encoding       original   True        0.3  0.02         0.538496   
2   phase-encoding       original  False        0.3  0.10         0.538496   
3   phase-encoding       original   True        0.3  0.10         0.538496   
4   phase-encoding       original  False        0.5  0.02         0.538496   
..             ...            ...    ...        ...   ...              ...   
75  phase-encoding    angleradius   True        0.8  0.10         0.538496   
76  phase-encoding    angleradius  False        0.9  0.02         0.538496   
77  phase-encoding    angleradius   True        0.9  0.02         0.538496   
78  phase-encoding    angleradius  False        0.9  0.10         0.538496   
79  phase-encoding    angleradius   True        0.9  0.10         0.538496   

                                       neuron_outputs  \
0   [0.7784423828125, 0.7755126953125, 0.778808593...   
1   [0.767333984375, 0.774658203125, 0.76953125, 0...   
2   [0.77587890625, 0.77099609375, 0.7752685546875...   
3   [0.77783203125, 0.76416015625, 0.773193359375,...   
4   [0.78369140625, 0.773193359375, 0.77197265625,...   
..                                                ...   
75  [0.897216796875, 0.2857666015625, 0.8939208984...   
76  [0.6729736328125, 0.674560546875, 0.0802001953...   
77  [0.9134521484375, 0.8663330078125, 0.911254882...   
78  [0.6748046875, 0.674560546875, 0.9952392578125...   
79  [0.9970703125, 0.5484619140625, 0.997192382812...   

                                         circuit_size  
0   [13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...  
1   [32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 3...  
2   [13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...  
3   [32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 3...  
4   [13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...  
..                                                ...  
75  [44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 4...  
76  [32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 3...  
77  [44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 4...  
78  [32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 3...  
79  [44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 4...  

[80 rows x 8 columns]

In [48]:
time_exec

[52.248915672302246, 52.951186180114746, 52.81357407569885, 58.142752170562744]

### Save Results

In [49]:
experiments = pd.concat([experiment_hsgs, experiment_phase])

In [50]:
experiments.to_csv('data_and_results/XOR/experiments_XOR_size.csv')

In [ ]:
xore = pd.read_csv('data_and_results/XOR/experiments_XOR_encoding_probs.csv')
xor = pd.read_csv('data_and_results/XOR/experiments_XOR_probs.csv')

In [ ]:
experiments = pd.concat([xor, xore])

In [ ]:
experiments.to_csv('data_and_results/XOR/experiments_XOR_probs.csv', index=False)